In [2]:
#기본세팅 - 데이터 획득 준비 작업
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

#matplotlib 한글폰트 로딩
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
     path = "c:/Windows/Fonts/malgun.ttf"
     font_name = font_manager.FontProperties(fname=path).get_name()
     rc('font', family=font_name)
else:
     print('Unknown system... sorry~~~~')
      
plt.rcParams['axes.unicode_minus'] = False

In [3]:
from selenium import webdriver
import time

In [4]:
driver = webdriver.Chrome('../driver/chromedriver')
driver.get("http://info.nec.go.kr/")

In [7]:
#frame 이동이 필요한 상황
driver.switch_to_default_content() #상위 프레임으로 돌아가는 명령, switch_to_frame사용 중 위치 혼동으로 인한 오류시 백업
driver.switch_to_frame('main')

In [8]:
#투개표 xpath
driver.find_element_by_xpath("""//*[@id="header"]/ul[1]/li[2]/a""").click()

In [12]:
driver.find_element_by_xpath("""//*[@id="presubmu"]/li[3]/a""").click()

In [14]:
#python보고 xpath찾게하기
make_xpath = """//*[contains(text(), "개표현황(읍면동별)")]"""
driver.find_elements_by_xpath(make_xpath)[1].click()

In [15]:
driver.find_element_by_xpath("""//*[@id="electionType1"]""").click()

In [16]:
election_list_raw = driver.find_element_by_xpath("""//*[@id="electionName"]""")
election_list = election_list_raw.find_elements_by_tag_name("option")
election_names_values = [option.text for option in election_list]
election_names_values = election_names_values[1:]
election_names_values

['제19대', '제18대', '제17대']

In [17]:
element = driver.find_element_by_id("electionName")
element.send_keys(election_names_values[0])

In [18]:
#option도 몇개 없고 귀찮으니 그냥 입력 날리기
element = driver.find_element_by_id("electionCode")
element.send_keys("대통령선거")

In [19]:
#시도정보 가져오기
sido_list_raw = driver.find_element_by_xpath("""//*[@id="cityCode"]""")
sido_list = sido_list_raw.find_elements_by_tag_name("option")
sido_names_values = [option.text for option in sido_list]
sido_names_values = sido_names_values[1:]
sido_names_values

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

In [20]:
#서울선택
element = driver.find_element_by_id("cityCode")
element.send_keys(sido_names_values[0])

In [21]:
#구시군 정보 가져오기
sigun_list_raw = driver.find_element_by_xpath("""//*[@id="townCode"]""")
sigun_list = sigun_list_raw.find_elements_by_tag_name("option")
sigun_names_values = [option.text for option in sigun_list]
sigun_names_values = sigun_names_values[1:]
sigun_names_values

['종로구',
 '중구',
 '용산구',
 '성동구',
 '광진구',
 '동대문구',
 '중랑구',
 '성북구',
 '강북구',
 '도봉구',
 '노원구',
 '은평구',
 '서대문구',
 '마포구',
 '양천구',
 '강서구',
 '구로구',
 '금천구',
 '영등포구',
 '동작구',
 '관악구',
 '서초구',
 '강남구',
 '송파구',
 '강동구']

In [22]:
#구시군 정보 전송
element = driver.find_element_by_id("townCode")
element.send_keys(sigun_names_values[0])

In [27]:
#전송버튼 클릭
driver.find_element_by_xpath("""//*[@id="spanSubmit"]""").click()

In [28]:
from bs4 import BeautifulSoup

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [29]:
tmp = soup.find_all('td', 'alignR')
tmp[1:5]

[<td class="alignR">102,566</td>,
 <td class="alignR">42,512</td>,
 <td class="alignR">22,325</td>,
 <td class="alignR">22,313</td>]

In [44]:
# 19대 대선 개표 결과 데이터 획득하기
sido_name_list = []
sigun_name_list = []

for sido_value in tqdm_notebook(sido_names_values):    
    element = driver.find_element_by_id("cityCode")
    element.send_keys(sido_value)
    
    time.sleep(1)
    
    sigun_list_raw = driver.find_element_by_xpath("""//*[@id="townCode"]""")
    sigun_list = sigun_list_raw.find_elements_by_tag_name("option")
    sigun_names_values = [option.text for option in sigun_list]
    sigun_names_values = sigun_names_values[1:]
    
    for sigun_value in sigun_names_values:
        sido_name_list.append(sido_value)
        sigun_name_list.append(sigun_value)
    

HBox(children=(IntProgress(value=0, max=17), HTML(value='')))

In [45]:
election_result = pd.DataFrame({'광역시도':sido_name_list, '시군':sigun_name_list})

election_result.head()

,광역시도,시군
0,서울특별시,종로구
1,서울특별시,중구
2,서울특별시,용산구
3,서울특별시,성동구
4,서울특별시,광진구
